In [180]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [181]:
data = pd.read_csv('NED_DATA.csv', skiprows=12)#,dtype=float,skiprows=12)
data.head(8) #showing first 8 values bc that is where the first NaN appears in 'Hubble const.'

,FRN,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,Unnamed: 10,Unnamed: 11,Hubble const.,Adopted LMC modulus,Date (Yr. - 1980)
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [182]:
#Need to fix the errors in column headers

data.columns = data.columns[1:].tolist()+['Notes']
data.head(8)

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,Unnamed: 10,Unnamed: 11,Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [183]:
data.rename(columns={'Unnamed: 11': 'redshift (z)', 'Unnamed: 10': 'SNID'}, inplace=True)
data.head(8)
#Now we fixed the dataframe

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,SNID,redshift (z),Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [184]:
#I'm first going to drop columns that do not help such as Exclusion code, REFCODE, SNID, Adopted LMC modulus and Notes
'''reason for dropping each
Exclusion code - Tells us if the data is from sources incorporated with NED which is not going to tell us about the measurement
D - We want our decision tree to be based on methods and results not the actual objects
G - Same as D they both use a number to describe the object
REFCODE - Tells what paper the measurement is from
SN_ID - Only relevant for supernova measurements which is not all of our datapoints
Adopted LMC modulus - Similar to SNID only relevant to certain measurements
Notes - Extra infor about measurement not present in most
'''
data = data.drop(['Exclusion Code','D','G','REFCODE','SNID','Adopted LMC modulus','Notes'], axis=1)
#We are looking at whether or not there is a hubble const measurement so we don't want to remove those
data['Hubble const.'] = data['Hubble const.'].fillna(0.0)
data.head(8)

,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980)
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
4,2MASX J00000138+1530350,38.86,0.46,592.0,FP,NaN,70.0,36.0
5,2MASX J00000155-0929403,39.79,0.46,910.0,FP,NaN,70.0,36.0
6,UGC 12889,33.65,0.66,53.7,Tully-Fisher,NaN,103.0,4.0
7,UGC 12889,34.16,0.40,68.0,Tully-Fisher,NaN,0.0,27.0


In [185]:
data = data.dropna()
hubble = data['Hubble const.']
print(data.shape)
print(hubble.shape)
data.head(8)

(30116, 8)
(30116,)


,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980)
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
16,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0
17,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0
21,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0
22,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0


In [186]:
print(hubble.unique())
#there is an odd value coming out here that is 69.7 100. maybe its just printing weirdly
for x in hubble.unique():
    print(x)
#yes just printing weirdly on my screen

[ 70.   74.2   0.   72.   73.8  65.   60.   73.   74.   69.7 100.   71.
  70.8  71.6]
70.0
74.2
0.0
72.0
73.8
65.0
60.0
73.0
74.0
69.7
100.0
71.0
70.8
71.6


In [187]:
#I want to have a column that is either 1 or 0 for 'has hubble measurement or doesnt. So i realize I didn't really need to pull out this 'hubble' variable
hubble2 = np.copy(hubble) #It was modifying the Hubble const column in the df
hubble2[hubble2 != 0.0] = 1.0
#Then stick it back in the df
data['Has Hubble'] = hubble2
data.head(8)

,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980),Has Hubble
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0,1.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0,1.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0,1.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0,1.0
16,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0,1.0
17,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0,1.0
21,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0,1.0
22,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0,1.0


This concludes cleaning the data

In [188]:
#Attempt at using ChatGPT to build the tree
#It suggests dropping the columns GalaxyID and Hubble const. both of which I had considered dropping
#Galaxy ID makes sense to drop because it can simply be used as an identifier for the measurement
#Hubble const. makes sense to drop because it is what we are trying to predict
data = data.drop(columns=["Galaxy ID", "Hubble const."])
#It also wants to one hot encode the method column which is fine because there are not a crazy amount of differnt methods
print(data['Method'].unique())
data["Method"] = data["Method"].astype("category").cat.codes
#Lastly split into Target and feature
X = data.drop(columns=["Has Hubble"]).values
y = data["Has Hubble"].values
print(X.shape, y.shape) #just checking and it looks alright


['SNIa SDSS' 'SNIa' 'SNII optical' 'BCG' 'GRB' 'HII LF' 'D-Sigma'
 'Faber-Jackson' 'G Lens']
(30116, 6) (30116,)


In [189]:
#Create a function for gini impurity
def gini(y):
    """Calculate Gini impurity for a list of labels.
    input:
        y: targets, list
    output:
        value of the gini impurity for the input target
    """
    classes, counts = np.unique(y, return_counts=True) #Gives all the unique values as well as the counts of them (how many times they appear)
    prob_sq = (counts / counts.sum()) ** 2 #probability (events/total) of each unique value^2
    return 1 - prob_sq.sum() #computes 1-sum of the prob sq

In [201]:
def best_split(X, y):
    """Find the best feature and threshold to split on. (This will be a greedy tree)
    input:
        X: features, lists containing all the features and their datapoints
        y: target, list of the target (1/0)
    output:
        best_feature: the best feature to split on (int used as an index)
        best_threshold: the threshold to split the feature on (float)
    """
    #set baseline for gini and initialize feature/threshold
    best_gini = 1
    best_feature = None
    best_threshold = None

    n_samples, n_features = X.shape # create sample and feature sizes

    for feature in range(n_features):
        thresholds = np.unique(X[:, feature]) #finds all unique thresholds we could have for each feature (seems like a lot of computation)

        for threshold in thresholds: #on each threshold previously defined
            left_mask = X[:, feature] <= threshold #Splits the feature on one threshold at a time
            right_mask = ~left_mask #other side of the threshold is opposite of first side (left and right are a bit confusing to use in description)

            if left_mask.sum() == 0 or right_mask.sum() == 0: #if either side of the split is empty we skip this threshold
                continue
#            print(left_mask)
            left_gini = gini(y[left_mask]) #compute gini impurity on the left and right
            right_gini = gini(y[right_mask])
            weighted_gini = (left_mask.sum() * left_gini + right_mask.sum() * right_gini) / n_samples #Calculate a weighted gini adds weighted left and right
            #For left you do left_gini * total on left/total samples (total on the left works because )

            if weighted_gini < best_gini: #We want to choose the threshold with the lowest gini impurity (best sorted)
                #If we have a lower impurity at this threshold then we reinitialize our 'best'
                best_gini = weighted_gini
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        #Class that stores all the info on the node
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # Leaf node value

def build_tree(X, y, depth=0, max_depth=5):
    '''
    Function that creates the tree
    input:
        X: features, lists containing all the features and their datapoints
        y: target, list of the target (1/0)
        depth: what part of the tree are we looking at (int)
        max_depth: what is the maximum depth we allow the tree to be (int)
    '''
    if len(set(y)) == 1 or depth >= max_depth: #Important argument If we reached the max depth or our leaf only has 1 y value we continue.
        leaf_value = np.bincount(y.astype(int)).argmax() #defines the most common value on this leaf
        return Node(value=leaf_value) #returns the node values

    feature, threshold = best_split(X, y) #calculates the best possible split if the tree hasn't 'ended'
    if feature is None: #This is kind of silly to have because likely the last if statement would have caught the end of the tree but still useful backup
        leaf_value = np.bincount(y.astype(int)).argmax() 
        return Node(value=leaf_value)

    left_mask = X[:, feature] <= threshold #Just like before we split on the threshold into left and right
    right_mask = ~left_mask

    #Creates the recursive part of the function that builds more 'branches' and 'leaves' on our tree
    left = build_tree(X[left_mask], y[left_mask], depth + 1, max_depth) #Now we continue building the tree from the left and right 
    right = build_tree(X[right_mask], y[right_mask], depth + 1, max_depth)

    return Node(feature=feature, threshold=threshold, left=left, right=right)

In [ ]:
def predict_tree(node, sample):
    '''
    This function pretty much just spits out what the value is at each leaf
    input:
        node: information about the tree such as thresholds and features to split on from the build tree function
        sample: a feature which is used to make the predcition using the splits from the tree
    output:
        node.value: the value of that node, either 1 or 0 for has or doesn't have hubble value
    '''
    while node.value is None: #pretty much saying before we have any nodes we will make some
        if sample[node.feature] <= node.threshold: #check if the feature (sample) is on the left or right of this threshold 
            node = node.left
        else:
            node = node.right
    return node.value

In [ ]:
tree = build_tree(X, y, max_depth=5) #Build the tree
predictions = [predict_tree(tree, sample) for sample in X] #Use the tree 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 